In [23]:
import pandas as pd

# Load the dataset to check its contents
file_path = './lakik.csv'
data = pd.read_csv(file_path, encoding='utf-8')

# Display the first few rows of the dataset to understand its structure
data.head()


,location,property_type,price,area,rooms,condition,year_of_construction,comfort,floor,building_levels,...,price_range_bottom,price_range_top,area_range,area_range_bottom,area_range_top,developer,nearby_universities,nearby_schools,public_transport_nearby,historical_note
0,"Budapest XIX. kerület, Báthory utca",Eladó tégla lakás,79600000.0,73.0,3.0,újszerű,2019.0,duplakomfortos,0.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"42-es villamos, M3-as metró",NaN
1,"Budapest IV. kerület, Újpest",Eladó tégla lakás,78230000.0,68.0,3.0,új építésű,2023.0,összkomfortos,2.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,kiváló tömegközlekedési lehetőség,NaN
2,"Budapest XIV. kerület, Dorozsmai utca",Eladó tégla lakás,139800000.0,90.0,4.0,újszerű,2018.0,duplakomfortos,5.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"tömegközlekedés (villamos, troli, busz) 5-10 p...",NaN
3,"Budapest V. kerület, Zrínyi utca",Eladó tégla lakás,2360000.0,1.0,2.0,felújított,NaN,NaN,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Budapest V. kerület, Károly körút 26.",Eladó tégla lakás,109000000.0,69.0,2.0,jó állapotú,NaN,összkomfortos,4.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"M1, M2, M3 metróvonalak, 72-es trolibusz, szám...",NaN


In [24]:
def clean_and_transform_data(df):
    df = df[df['price'] >= 20_000_000]
    df['air_conditioning'] = df['air_conditioning'].fillna(False)
    df['accessible'] = df['accessible'].fillna(False)
    df = df[df['area'] >= 10]
    df[['district', 'street']] = df['location'].str.split(',', expand=True)
    return df

data = clean_and_transform_data(data)

C:\Users\takat\AppData\Local\Temp\ipykernel_14776\2593839739.py:5: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['air_conditioning'] = df['air_conditioning'].fillna(False)
C:\Users\takat\AppData\Local\Temp\ipykernel_14776\2593839739.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['air_conditioning'] = df['air_conditioning'].fillna(False)
C:\Users\takat\AppData\Local\Temp\ipykernel_14776\2593839739.py:6: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future ver

In [26]:
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv('GOOGLE_MAPS_API_KEY')

In [28]:
import requests
import pandas as pd
from tqdm import tqdm

def get_lat_long(address, api_key):
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"
    params = {"address": address, "key": api_key}
    
    try:
        response = requests.get(base_url, params=params, timeout=10)  # Adding timeout
        response.raise_for_status()  # Raise an HTTPError for bad responses
        geocode_data = response.json()

        if geocode_data['status'] == 'OK':
            location = geocode_data['results'][0]['geometry']['location']
            return location['lat'], location['lng']
        else:
            return None, None  # Return None in case of issues like ZERO_RESULTS
    except requests.exceptions.RequestException as e:
        print(f"Error occurred for address '{address}': {e}")
        return None, None

tqdm.pandas()
data['lat'], data['lng'] = zip(*data['location'].progress_apply(lambda x: get_lat_long(x, api_key)))
print(data.head())


 47%|████▋     | 1003/2112 [06:02<1:01:06,  3.31s/it]

Error occurred for address 'Budapest III. kerület, Kiscelli utca': HTTPSConnectionPool(host='maps.googleapis.com', port=443): Read timed out. (read timeout=10)


100%|██████████| 2112/2112 [12:24<00:00,  2.84it/s]  

                                location      property_type        price  \
0    Budapest XIX. kerület, Báthory utca  Eladó tégla lakás   79600000.0   
1           Budapest IV. kerület, Újpest  Eladó tégla lakás   78230000.0   
2  Budapest XIV. kerület, Dorozsmai utca  Eladó tégla lakás  139800000.0   
4  Budapest V. kerület, Károly körút 26.  Eladó tégla lakás  109000000.0   
5   Budapest XI. kerület, Bercsényi utca  Eladó tégla lakás   99980000.0   

   area  rooms    condition  year_of_construction         comfort  floor  \
0  73.0    3.0      újszerű                2019.0  duplakomfortos    0.0   
1  68.0    3.0   új építésű                2023.0   összkomfortos    2.0   
2  90.0    4.0      újszerű                2018.0  duplakomfortos    5.0   
4  69.0    2.0  jó állapotú                   NaN   összkomfortos    4.0   
5  95.0    3.0  felújítandó                1950.0             NaN    3.0   

   building_levels  ... developer nearby_universities nearby_schools  \
0             

In [29]:
# data.to_csv('./tiszta_lakik.csv')

In [55]:
data = pd.read_csv('./tiszta_lakik.csv')

In [57]:
data[['lat', 'lng', 'location']] = data[['lat', 'lng', 'location']].dropna()



In [61]:
# Assuming the DataFrame `data` contains the necessary columns.
# Saving the specified columns without applying dropna.

# Select only the required columns
data_to_save = data[['location', 'lat', 'lng', 'price', 'area']]

# Save the data to a CSV file
data_to_save.to_csv('tiszta_lakik_kepler.csv', index=False)

print("Data has been saved as 'tiszta_lakik_kepler.csv'")

data

Data has been saved as 'tiszta_lakik_kepler.csv'


,Unnamed: 0,location,property_type,price,area,rooms,condition,year_of_construction,comfort,floor,...,developer,nearby_universities,nearby_schools,public_transport_nearby,historical_note,district,street,price_per_area,lat,lng
0,0,"Budapest XIX. kerület, Báthory utca",Eladó tégla lakás,79600000.0,73.0,3.0,újszerű,2019.0,duplakomfortos,0.0,...,NaN,NaN,NaN,"42-es villamos, M3-as metró",NaN,Budapest XIX. kerület,Báthory utca,1.090411e+06,47.452671,19.138008
1,1,"Budapest IV. kerület, Újpest",Eladó tégla lakás,78230000.0,68.0,3.0,új építésű,2023.0,összkomfortos,2.0,...,NaN,NaN,NaN,kiváló tömegközlekedési lehetőség,NaN,Budapest IV. kerület,Újpest,1.150441e+06,47.564886,19.091309
2,2,"Budapest XIV. kerület, Dorozsmai utca",Eladó tégla lakás,139800000.0,90.0,4.0,újszerű,2018.0,duplakomfortos,5.0,...,NaN,NaN,NaN,"tömegközlekedés (villamos, troli, busz) 5-10 p...",NaN,Budapest XIV. kerület,Dorozsmai utca,1.553333e+06,47.527347,19.102821
3,4,"Budapest V. kerület, Károly körút 26.",Eladó tégla lakás,109000000.0,69.0,2.0,jó állapotú,NaN,összkomfortos,4.0,...,NaN,NaN,NaN,"M1, M2, M3 metróvonalak, 72-es trolibusz, szám...",NaN,Budapest V. kerület,Károly körút 26.,1.579710e+06,47.496117,19.057441
4,5,"Budapest XI. kerület, Bercsényi utca",Eladó tégla lakás,99980000.0,95.0,3.0,felújítandó,1950.0,NaN,3.0,...,NaN,NaN,NaN,NaN,NaN,Budapest XI. kerület,Bercsényi utca,1.052421e+06,47.476683,19.050764
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2107,2118,"Budapest XIII. kerület, Népfürdő utca",Eladó tégla lakás,59900000.0,38.0,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,A 15-ös autóbusz és a 79-es trolibusz a ház el...,NaN,Budapest XIII. kerület,Népfürdő utca,1.576316e+06,47.534000,19.059349
2108,2119,"Budapest III. kerület, Pacsirtamező utca 20.",Eladó tégla lakás,99800000.0,103.0,4.0,felújított,NaN,duplakomfortos,4.0,...,NaN,NaN,NaN,"Busz: 9, 29, 111; Villamos: 17, 19, 41; Éjszak...",NaN,Budapest III. kerület,Pacsirtamező utca 20.,9.689320e+05,47.534259,19.039703
2109,2120,"Budapest IX. kerület, Márton utca 7-9.",Eladó tégla lakás,75000000.0,46.0,2.0,NaN,2022.0,összkomfortos,5.0,...,NaN,NaN,NaN,"Villamos: 2B, 23, 24, 51, 51A; Busz: 281",NaN,Budapest IX. kerület,Márton utca 7-9.,1.630435e+06,47.477487,19.078942
2110,2121,"Budapest IX. kerület, Belső-Ferencváros",Eladó tégla lakás,44900000.0,45.0,1.0,felújított,NaN,összkomfortos,0.0,...,NaN,NaN,NaN,NaN,NaN,Budapest IX. kerület,Belső-Ferencváros,9.977778e+05,47.485003,19.064913


In [47]:
import folium
import pandas as pd

budapest_map = folium.Map(location=[47.4979, 19.0402], zoom_start=12)

# Plot each point on the map
for index, row in data.iterrows():
    if row['location']:
        folium.Marker(
            location=[row['lat'], row['lng']],
            popup=row['location'],  # Display the location name as a popup
        ).add_to(budapest_map)

budapest_map.save('budapest_map.html')
